In [ ]:
%%writefile phonebook_search.cu
#include <bits/stdc++.h>

using namespace std;

ifstream phonebook("/content/phonebook.txt");

string getInput() {
  char ch;
  string str;
  bool reading = false;
  while (phonebook.get(ch)) {
    if (reading) {
      if (ch == '"') {
        break;
      }

      str.push_back(ch);
    } else if (ch == '"') {
      reading = true;
    }
  }
  return str;
}

struct Contact {
  char name[65];
  char number[15];
};

vector<Contact> prepareContactList() {
  vector<Contact> contactList;
  while (phonebook.peek() != EOF) {
    Contact contact;
    strcpy(contact.name, getInput().c_str());
    strcpy(contact.number, getInput().c_str());
    contactList.push_back(contact);
  }
  return contactList;
}

__device__ bool search(char *text, const char *term) {
  int m = 0, n = 0;
  while (text[m]) m++;
  while (term[n]) n++;
  if (m < n) {
    return false;
  }

  for (int i = 0; i < m-n+1; i++) {
    bool matched = true;
    for (int j = 0; j < n; j++) {
      if (text[i+j] != term[j]) {
        matched = false;
        break;
      }
    }
    if (matched) {
      return true;
    }
  }
  return false;
}

__global__ void kernel(char *d_term, Contact *d_contacts, int offset) {
  if (search(d_contacts[threadIdx.x+offset].name, d_term)) {
    printf("Kernel %d %s %s\n", threadIdx.x + offset, d_contacts[threadIdx.x+offset].name, d_contacts[threadIdx.x+offset].number);
  }
}

int main(int argc, char* argv[]) {
  if (argc < 3) {
    cout << "Too few arguments." << endl;
    return 1;
  }
  vector<Contact> contactList = prepareContactList();
  int n = contactList.size();
  int threads = atoi(argv[1]);
  string term = "";
  for (int i = 2; i < argc; i++) {
    if (i != 2) {
      term += " ";
    }
    term += argv[i];
  }

  /*
  for (Contact contact : contactList) {
    if (search(contact.name, term.c_str())) {
      cout << contact.name << ' ' << contact.number << endl;
    }
  }
  */

  char *d_term;
  cudaMalloc(&d_term, sizeof(term));
  cudaMemcpy(d_term, (char *)term.c_str(), sizeof(term), cudaMemcpyHostToDevice);

  Contact *d_contacts;
  cudaMalloc(&d_contacts, n * sizeof(Contact));
  cudaMemcpy(d_contacts, contactList.data(), n * sizeof(Contact), cudaMemcpyHostToDevice);

  int remaining = n;
  while (remaining) {
    int tLimit = min(threads, remaining);
    kernel<<<1,tLimit>>>(d_term, d_contacts, n-remaining);
    remaining -= tLimit;
  }
  cudaDeviceSynchronize();

  return 0;
}

Overwriting phonebook_search.cu


In [ ]:
!nvcc -arch=sm_75 phonebook_search.cu -o phonebook_search

In [ ]:
!time ./phonebook_search 5 ri

Kernel 3 Ami Pari Na 4444444
Kernel 1 Alex Hari Son 01959041234
Kernel 2 Hiya Afrin 01888
Kernel 6 Ami Pari Na 4444444
Kernel 9 Ami Pari Na 4444444
Kernel 5 Alex Hari Son 01959041234
Kernel 8 Alex Hari Son 01959041234
Kernel 12 Ami Pari Na 4444444
Kernel 11 Alex Hari Son 01959041234
Kernel 14 Alex Hari Son 01959041234
Kernel 15 Ami Pari Na 4444444

real	0m0.113s
user	0m0.015s
sys	0m0.096s
